# L1: NLP tasks with a simple interface 🗞️

Load your HF API key and relevant Python libraries.

## mostly, using 1) summarization (facebook/bart-large-cnn), 2) text entity detection (dslim/bert-base-NER) tools (hf models) to summarize and identify entity and use UI tools to interact with people.

## did not use the gradio UI part for dslim/bert-base-NER, as did not use the AWS endpoint.

# Small specialist model

- Designed for a specific task
- Similar performance as a general purpose LLM
- Cheaper and faster to run compared to a general purpose LLM


- Summarize text and name them to recommendation  

## mainly using facebook bart-large-cnn

In [3]:
# !pip install python-dotenv

In [7]:
import os
import io
from IPython.display import Image, display, HTML
from PIL import Image
import base64
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
hf_api_key = os.environ['HF_API_KEY']
# hf_api_key = hf_api_key
hf_api_summary_base = 'https://api-inference.huggingface.co/models/facebook/bart-large-cnn'
# 'https://api-inference.huggingface.co/models/dslim/bert-base-NER'

In [9]:
# Helper function
import requests, json

#Summarization endpoint
def get_completion(inputs, parameters=None,ENDPOINT_URL=hf_api_summary_base):
    headers = {
      "Authorization": f"Bearer {hf_api_key}",
      "Content-Type": "application/json"
    }
    data = { "inputs": inputs }
    if parameters is not None:
        data.update({"parameters": parameters})
    response = requests.request("POST",
                                ENDPOINT_URL, headers=headers,
                                data=json.dumps(data)
                               )
    return json.loads(response.content.decode("utf-8"))

### How about running it locally?
The code would look very similar if you were running it locally instead of from an API. The same is true for all the models in the rest of the course, make sure to check the [Pipelines](https://huggingface.co/docs/transformers/main_classes/pipelines) documentation page

```py
from transformers import pipeline

get_completion = pipeline("summarization", model="shleifer/distilbart-cnn-12-6")

def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']
    
```

## Building a text summarization app

In [10]:
text = ('''The tower is 324 metres (1,063 ft) tall, about the same height
        as an 81-storey building, and the tallest structure in Paris.
        Its base is square, measuring 125 metres (410 ft) on each side.
        During its construction, the Eiffel Tower surpassed the Washington
        Monument to become the tallest man-made structure in the world,
        a title it held for 41 years until the Chrysler Building
        in New York City was finished in 1930. It was the first structure
        to reach a height of 300 metres. Due to the addition of a broadcasting
        aerial at the top of the tower in 1957, it is now taller than the
        Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the
        Eiffel Tower is the second tallest free-standing structure in France
        after the Millau Viaduct.''')

get_completion(text)

[{'summary_text': 'The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building. Its base is square, measuring 125 metres (410 ft) on each side. It is the second tallest free-standing structure in France after the Millau Viaduct.'}]

### Getting started with Gradio `gr.Interface`

In [13]:
# !pip install gradio

In [15]:
import gradio as gr
def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']

gr.close_all()
demo = gr.Interface(fn=summarize, inputs="text", outputs="text")
demo.launch(share=True, server_port=int(8070))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b4200338f0b9768688.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


`demo.launch(share=True)` lets you create a public link to share with your team or friends.

In [21]:
import gradio as gr

def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']

gr.close_all()
demo = gr.Interface(fn=summarize,
                    inputs=[gr.Textbox(label="Text to summarize", lines=6)],
                    outputs=[gr.Textbox(label="Result", lines=3)],
                    # you can modify how long the summary would be, here lines = 3
                    title="Text summarization with distilbart-cnn",
                    description="Summarize any text using the `shleifer/distilbart-cnn-12-6` model under the hood!"
                   )
demo.launch(share=True, server_port=int(8060))

Closing server running on port: 8060
Closing server running on port: 8070
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2148a4215ce5c232a8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# Bert Model

### Bert is a Machine Learning model for natural language processing.
### When parsing a text, it is useful to identify specific entities such as persons, companies, places, etc.
### To do so, we are using bert-base-NER, a 108 M parameter fine-tuned BERT model taht is ready to use for Named Entity Recognition (NER)
## Bert-base-NER has been trained to recognize four types of entities: localtion (LOC), organizations (ORG), person(PER), and Miscellaneous (MISC).

## Building a Named Entity Recognition app

We are using this [Inference Endpoint](https://huggingface.co/inference-endpoints) for `dslim/bert-base-NER`, a 108M parameter fine-tuned BART model on the NER task.

### How about running it locally?

```py
from transformers import pipeline

get_completion = pipeline("ner", model="dslim/bert-base-NER")

def ner(input):
    output = get_completion(input)
    return {"text": input, "entities": output}
    
```

In [50]:
hf_api_summary_base

'https://api-inference.huggingface.co/models/facebook/bart-large-cnn'

In [52]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
text = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(text)
print(ner_results)

# object detection in sentences.

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity': 'B-PER', 'score': 0.9990139, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.999645, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


In [61]:
# make it neat format
import json
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

text = "Wolfgan was born in Berlin. He was a prolific and influential composer of Classical period. \
He had a short life, but composed more than 800 works, representing virtually every Western classical genre of his time. \
Many of these compositions are acknowledged as pinnacles of the symphonic, concertante, chamber, operatic, and choral repertoire. \
Mozart is widely regarded as one of the greatest composers in the history of Western music,with his music admired for its \
'melodic beauty, its formal elegance and its richness of harmony and texture'."

ner_results = nlp(text)

# change float32 values to Python native floats for JSON serialization, important
for result in ner_results:
    result['score'] = float(result['score'])

# neat format
json_format= json.dumps(ner_results, indent=4, sort_keys=True)
print(json_format)


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[
    {
        "end": 4,
        "entity": "B-PER",
        "index": 1,
        "score": 0.9985072016716003,
        "start": 0,
        "word": "Wolf"
    },
    {
        "end": 7,
        "entity": "I-PER",
        "index": 2,
        "score": 0.8715917468070984,
        "start": 4,
        "word": "##gan"
    },
    {
        "end": 26,
        "entity": "B-LOC",
        "index": 6,
        "score": 0.9993138313293457,
        "start": 20,
        "word": "Berlin"
    },
    {
        "end": 83,
        "entity": "B-MISC",
        "index": 16,
        "score": 0.9977426528930664,
        "start": 74,
        "word": "Classical"
    },
    {
        "end": 183,
        "entity": "B-MISC",
        "index": 35,
        "score": 0.9996920824050903,
        "start": 176,
        "word": "Western"
    },
    {
        "end": 349,
        "entity": "B-PER",
        "index": 68,
        "score": 0.997958779335022,
        "start": 343,
        "word": "Mozart"
    },
    {
        "end": 

In [62]:
# make it neat format
import json
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

text = "Wolfgan was born in Berlin. He was a composer. \
He composed more than 800 works. \
Mozart is regarded as one of the greatest composers in the history of Western music. \
He visited Vienna in 1781."

ner_results = nlp(text)

# change float32 values to Python native floats for JSON serialization, important
for result in ner_results:
    result['score'] = float(result['score'])

# neat format
json_format= json.dumps(ner_results, indent=4, sort_keys=True)
print(json_format)


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[
    {
        "end": 4,
        "entity": "B-PER",
        "index": 1,
        "score": 0.9987508058547974,
        "start": 0,
        "word": "Wolf"
    },
    {
        "end": 7,
        "entity": "I-PER",
        "index": 2,
        "score": 0.8392647504806519,
        "start": 4,
        "word": "##gan"
    },
    {
        "end": 26,
        "entity": "B-LOC",
        "index": 6,
        "score": 0.9993405938148499,
        "start": 20,
        "word": "Berlin"
    },
    {
        "end": 86,
        "entity": "B-PER",
        "index": 20,
        "score": 0.9987033605575562,
        "start": 80,
        "word": "Mozart"
    },
    {
        "end": 157,
        "entity": "B-MISC",
        "index": 33,
        "score": 0.9995831847190857,
        "start": 150,
        "word": "Western"
    },
    {
        "end": 182,
        "entity": "B-LOC",
        "index": 38,
        "score": 0.9995728135108948,
        "start": 176,
        "word": "Vienna"
    }
]


In [38]:
# not in use
# # API_URL = os.environ['HF_API_NER_BASE'] #NER endpoint
# API_URL = "https://n9mjb8bgycgo2pbs.us-east-1.aws.endpoints.huggingface.cloud" #NER endpoint
# text = "My name is Andrew, I'm building DeepLearningAI and I live in California"
# get_completion(text, parameters=None, ENDPOINT_URL= API_URL)
# # get_completion(text, parameters=None, ENDPOINT_URL= "https://n9mjb8bgycgo2pbs.us-east-1.aws.endpoints.huggingface.cloud")


#### gr.interface()
- Notice below that we pass in a list `[]` to `inputs` and to `outputs` because the function `fn` (in this case, `ner()`, can take in more than one input and return more than one output.
- The number of objects passed to `inputs` list should match the number of parameters that the `fn` function takes in, and the number of objects passed to the `outputs` list should match the number of objects returned by the `fn` function.

In [63]:
# not in use
# def ner(input):
#     output = get_completion(input, parameters=None, ENDPOINT_URL=API_URL)
#     return {"text": input, "entities": output}

# gr.close_all()
# demo = gr.Interface(fn=ner,
#                     inputs=[gr.Textbox(label="Text to find entities", lines=2)],
#                     outputs=[gr.HighlightedText(label="Text with entities")],
#                     title="NER with dslim/bert-base-NER",
#                     description="Find entities using the `dslim/bert-base-NER` model under the hood!",
#                     allow_flagging="never",
#                     #Here we introduce a new tag, examples, easy to use examples for your application
#                     examples=["My name is Andrew and I live in California", "My name is Poli and work at HuggingFace"])
# demo.launch(share=True, server_port=int(8050))

### Adding a helper function to merge tokens

In [64]:
# not in use
# def merge_tokens(tokens):
#     merged_tokens = []
#     for token in tokens:
#         if merged_tokens and token['entity'].startswith('I-') and merged_tokens[-1]['entity'].endswith(token['entity'][2:]):
#             # If current token continues the entity of the last one, merge them
#             last_token = merged_tokens[-1]
#             last_token['word'] += token['word'].replace('##', '')
#             last_token['end'] = token['end']
#             last_token['score'] = (last_token['score'] + token['score']) / 2
#         else:
#             # Otherwise, add the token to the list
#             merged_tokens.append(token)

#     return merged_tokens

# def ner(input):
#     output = get_completion(input, parameters=None, ENDPOINT_URL=API_URL)
#     merged_tokens = merge_tokens(output)
#     return {"text": input, "entities": merged_tokens}

# gr.close_all()
# demo = gr.Interface(fn=ner,
#                     inputs=[gr.Textbox(label="Text to find entities", lines=2)],
#                     outputs=[gr.HighlightedText(label="Text with entities")],
#                     title="NER with dslim/bert-base-NER",
#                     description="Find entities using the `dslim/bert-base-NER` model under the hood!",
#                     allow_flagging="never",
#                     examples=["My name is Andrew, I'm building DeeplearningAI and I live in California", "My name is Poli, I live in Vienna and work at HuggingFace"])

# demo.launch(share=True, server_port=int(8040))

In [65]:
gr.close_all()

Closing server running on port: 8040
Closing server running on port: 8060
Closing server running on port: 8050
Closing server running on port: 8060
Closing server running on port: 8070
Closing server running on port: 8050
Closing server running on port: 8070


## How to get your own Hugging Face API key (token)

Hugging Face "API keys" are called "User Access tokens".  

You can create your own User Access Tokens here: [Access Tokens](https://huggingface.co/settings/tokens).

#### Save your user access tokens to environment variables
To save your access token securely on your own machine:
- Create a `.env` file in the root directory of your project.
- Edit the file to contain the following:  
`HF_API_KEY="abc123"` replace that string with your user access token.
- Save the .env file.
- Install Python-dotenv, which allows you to run that first code cell at the top of this jupyter notebook:  
`pip install python-dotenv`


For more information on how to get your own access tokens, please see [User access tokens](https://huggingface.co/docs/hub/security-tokens#:~:text=To%20create%20an%20access%20token,you're%20ready%20to%20go!)